In [1]:
# import os
# import sys

# sys.dont_write_bytecode = True
# os.environ["NUMBA_DISABLE_JIT"] = "1"

In [29]:
import pandas as pd
import numpy as np
from quantfreedom.class_practice.enums import *
from quantfreedom.class_practice.base import backtest_df_only, backtest_sim_6, plot_one_result
from quantfreedom.class_practice.helper_funcs import create_os_cart_product_nb
from quantfreedom.indicators.talib_ind import from_talib



price_data = pd.read_hdf("../../tests/data/prices.hd5")
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
price_data['BTCUSDT']

candle_info,open,high,low,close
open_time,,,,
2020-10-01 00:00:00,"10,759.00","10,817.50","10,759.00","10,790.00"
2020-10-01 00:30:00,"10,790.00","10,790.50","10,775.00","10,777.00"
2020-10-01 01:00:00,"10,777.00","10,793.00","10,777.00","10,793.00"
2020-10-01 01:30:00,"10,793.00","10,836.00","10,792.50","10,822.00"
2020-10-01 02:00:00,"10,822.00","10,841.50","10,808.00","10,808.50"
...,...,...,...,...
2021-04-14 21:30:00,"62,703.50","62,812.50","62,615.50","62,795.00"
2021-04-14 22:00:00,"62,795.00","62,825.00","62,591.00","62,680.00"
2021-04-14 22:30:00,"62,680.00","62,993.00","62,680.00","62,849.50"


In [4]:
rsi_ind = from_talib(
    func_name='rsi',
    nickname='rsi',
    price_data=price_data,
    input_names = ['close'],
    parameters = {'timeperiod': [15, 20]}
)
entries = rsi_ind.is_below(
    user_args=[40, 50],
    candle_ohlc="close",
)

In [5]:
account_state = AccountState()
backtest_settings = BacktestSettings(
    divide_records_array_size_by=2,
    gains_pct_filter=10.0,
    total_trade_filter=10,
    upside_filter=0.0,
)
exchange_settings = ExchangeSettings()
order_settings_arrays = OrderSettingsArrays(
    risk_account_pct_size=np.array([1.0]) / 100,
    sl_based_on_add_pct=np.array([0.01, .02]) / 100,
    sl_based_on_lookback=np.array([50]),
    risk_reward=np.array([2.0, 5.0]),
    leverage_type=np.array([LeverageType.Dynamic]),
    sl_candle_body_type=np.array([CandleBodyType.Low]),
    increase_position_type=np.array([IncreasePositionType.RiskPctAccountEntrySize]),
    stop_loss_type=np.array([StopLossType.SLBasedOnCandleBody]),
    take_profit_type=np.array([TakeProfitType.RiskReward]),
    max_equity_risk_pct=np.array([3]) / 100,
    order_type=np.array([OrderType.Long]),
    sl_to_be_based_on_candle_body_type=np.array([CandleBodyType.Nothing]),
    sl_to_be_when_pct_from_candle_body=np.array([0.0]) / 100,
    sl_to_be_zero_or_entry_type=np.array([SLToBeZeroOrEntryType.Nothing]),
    trail_sl_based_on_candle_body_type=np.array([CandleBodyType.High]),
    trail_sl_when_pct_from_candle_body=np.array([3.0]) / 100,
    trail_sl_by_pct=np.array([1.0]) / 100,
    static_leverage=np.array([0.0]),
    tp_fee_type=np.array([TakeProfitFeeType.Limit]),
)

In [6]:
os_cart_arrays = create_os_cart_product_nb(order_settings_arrays=order_settings_arrays)

In [14]:
list(price_data.columns.levels[0])

['BTCUSDT', 'EOSUSD', 'ETHUSD', 'LTCUSDT', 'XRPUSD']

In [18]:
strat_res_df = backtest_df_only(
    account_state=account_state,
    os_cart_arrays=os_cart_arrays,
    backtest_settings=backtest_settings,
    exchange_settings=exchange_settings,
    price_data=price_data['EOSUSD'],
    entries=entries['EOSUSD'],
)

Starting the backtest now ... and also here are some stats for your backtest.

Total indicator settings to test: 4
Total order settings to test: 4
Total combinations of settings to test: 16

Total candles: 9,408
Total candles to test: 150,528


In [30]:
order_records_df = backtest_sim_6(
    account_state=account_state,
    os_cart_arrays=os_cart_arrays,
    exchange_settings=exchange_settings,
    price_data=price_data["EOSUSD"],
    entries=entries["EOSUSD"],
    strat_indexes=[0, 2, 4],
    strat_res_df=strat_res_df,
)

In [32]:
plot_one_result(
    strat_res_df=strat_res_df,
    strat_result_index=0,
    price_data=price_data["EOSUSD"],
    order_records_df=order_records_df,
)